In [5]:
from classes.Dataset_Basic import Dataset_Basic
from classes.Process_Stages import Process_Stages

from classes.Dataset_Settings import Dataset_Settings

from run_models.cosine_sililarity.classes.Dataset_Cosine import Dataset_Cosine
from run_models.gensim.classes.Process_Stages_Gensim import Process_Stages_Gensim

from run_models.embed_words.classes.Embed_Words import Embed_Words

from run_models.gensim.services.gensim_services import gensim_download, gensim_save, gensim_load
from word_embedding.models.classes.EmbeddingModel import EmbeddingModel

name_dataset = "conceptnet_cosine_similarity"

datasets = {}
datasets["neural_course"] = Dataset_Cosine(
    df_name="neural_course",
    model_name=name_dataset, # used for dir name inside data_saved

    language="english",

    datasets = {
        "emebed_words_conceptnet": Dataset_Settings(
            df=None,
            may_run_now=False,
            required=True,
            parquet=True
        ),
        name_dataset: Dataset_Settings(
            df=None,
            may_run_now=True,
            required=True,
            parquet=True,
            name_required_dataset="gensim"
        ),
    },

    columns_to_add = {name_dataset: {"cosine_similarity": []}},

)

# get dataset, process dataset, save dataset
datasets["neural_course"].get_dataset()
datasets["neural_course"].process_dataset()
datasets["neural_course"].add_columns()
datasets["neural_course"].save()


Found neural_course with type .parquet in (data_saved/emebed_words_conceptnet) and converted it to df
df found: True, name: emebed_words_conceptnet
File neural_course not found in (data_saved/conceptnet_cosine_similarity)
df found: False, name: conceptnet_cosine_similarity
No need to run emebed_words_conceptnet for neural_course
Running conceptnet_cosine_similarity for neural_course
basic processing starting for neural_course


  0%|          | 0/646 [00:00<?, ?it/s]/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 646/646 [00:00<00:00, 30564.94it/s]

no saving needed because basic emebed_words_conceptnet already done on neural_course
saving new conceptnet_cosine_similarity phase for: neural_course


# Regression testing

In [1]:
# run regression
from run_models.regressions.classses.Regression_Model import Regression_Model

# dataset
from classes.Dataset_Settings import Dataset_Settings

import pandas as pd

# import datasets
data = pd.read_parquet("data_saved/cosine_similarity_glove/beetle.parquet")
data.head(1)

# splitting data
seed = 42

dataset = Dataset_Settings(
    df=data, 
    may_run_now=True, 
    required=True
)

dataset.split_datasets(seed=seed)


In [2]:
data.head(2)

,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,normalized_points,cosine_similarity
0,0,What role does the path play in determining wh...,0,"[-0.3919050097465515, -0.9497899860143661, 0.2...","[-0.6870420165359974, -1.268229991197586, 0.86...",1,1,NaN,beetle,1.0,0.889654
1,1,What role does the path play in determining wh...,0,"[-0.7237350195646286, -0.9759199768304825, -0....","[-0.6870420165359974, -1.268229991197586, 0.86...",1,1,NaN,beetle,1.0,0.782875


In [3]:
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression, Ridge

isotonic_regression = Regression_Model(
    dataset=dataset,
    model=Ridge,
    x_column="cosine_similarity",
    y_column="normalized_points"
)


In [4]:
isotonic_regression.train()


In [5]:
isotonic_regression.test()


In [6]:
isotonic_regression.model_accuracty(True, "test_df")


Accuracy: 0.68202416918429

Weighted-Averaged Precision: 0.676948924357992
Weighted-Averaged Recall: 0.68202416918429
Weighted-Averaged F1-score: 0.6778041969179242


In [7]:
isotonic_regression["rmse_test"]
isotonic_regression["dataset"]["test_df"].head(200)


,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,normalized_points,cosine_similarity,y_pred,pred_points
4343,4343,Under what circumstances will a damaged/ burne...,104,"[-0.19539102166891098, -1.7355899810791016, -0...","[-0.5944120176136494, -1.3016799986362457, -0....",1,1,NaN,beetle,1.0,0.893038,0.560489,1.0
1561,1561,What role does the path play in determining wh...,71,"[-0.2501409873366356, -0.08860999345779419, -0...","[-0.6870420165359974, -1.268229991197586, 0.86...",0,1,NaN,beetle,0.0,0.298409,0.124440,0.0
6058,6058,As you move the multimeter leads from one bulb...,124,"[0.24106000363826752, -0.6620500087738037, -0....","[-0.08004009700380266, 0.24327020067721605, -0...",0,1,NaN,beetle,0.0,0.296574,0.123095,0.0
5202,5202,Explain why you got a voltage reading of 0 for...,114,"[-0.2537520898040384, 0.50274140201509, -1.189...","[-0.34443811303935945, 0.4292133990675211, -0....",0,1,NaN,beetle,0.0,0.956499,0.607025,1.0
5242,5242,Explain why you got a voltage reading of 0 for...,115,"[-0.7913239747285843, -0.024784604087471962, -...","[-0.2708380890544504, 0.37087140046060085, -0....",1,1,NaN,beetle,1.0,0.870173,0.543721,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2348,2348,Why does measuring voltage help you locate a b...,81,"[-0.04446506733074784, 0.7635231642052531, -0....","[0.36926503479480743, -0.0806449968367815, -0....",0,1,NaN,beetle,0.0,0.889065,0.557575,1.0
5201,5201,Explain why you got a voltage reading of 0 for...,114,"[-0.34443811303935945, 0.4292133990675211, -0....","[-0.34443811303935945, 0.4292133990675211, -0....",1,1,NaN,beetle,1.0,1.000000,0.638925,1.0
2847,2847,When switch X was closed and switch Y was open...,85,"[-0.4420109912753105, -0.7372259795665741, 0.9...","[-1.0937750115990639, -0.6824459731578827, -0....",0,1,NaN,beetle,0.0,0.755623,0.459720,0.0
2526,2526,Why was bulb A on when switch Y was open and s...,82,"[-0.02937900274991989, -1.237204011529684, -0....","[-1.0937750115990639, -0.6824459731578827, -0....",1,1,NaN,beetle,1.0,0.707393,0.424353,0.0


In [8]:
dataset["test_df"].describe()

,row_id,question_id,assigned_points,max_points,domain,normalized_points,cosine_similarity,y_pred,pred_points
count,1324.000000,1324.000000,1324.000000,1324.0,0.0,1324.000000,1324.000000,1324.000000,1324.000000
mean,3347.760574,89.564955,0.398036,1.0,NaN,0.398036,0.686223,0.408829,0.348943
std,1913.017269,27.509408,0.489678,0.0,NaN,0.489678,0.223594,0.163964,0.476816
min,12.000000,0.000000,0.000000,1.0,NaN,0.000000,-0.109285,-0.174527,-0.000000
25%,1705.000000,73.000000,0.000000,1.0,NaN,0.000000,0.558836,0.315414,0.000000
50%,3357.000000,91.000000,0.000000,1.0,NaN,0.000000,0.725997,0.437995,0.000000
75%,5024.000000,112.000000,1.000000,1.0,NaN,1.000000,0.864619,0.539648,1.000000
max,6609.000000,130.000000,1.000000,1.0,NaN,1.000000,1.000000,0.638925,1.000000


In [7]:
# model.model.index_to_key


In [8]:
# model.model[f"/c/en/open"]
